# Проект 5. Введение в машинное обучение

##  Рекомендация тарифов для клиентов оператора связи

Исходные данные - данные о поведении клиентов, которые уже перешли на эти тарифы. 

Задача - построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

## 1. Откройте и изучите файл

In [1]:
import pandas as pd
import numpy as np
from scipy import stats as st
from matplotlib import pyplot as plt
import math
from math import factorial
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

df = pd.read_csv('/datasets/users_behavior.csv')
df.head()


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


## 2. Разбейте данные на выборки

In [2]:
df_train, df_valid_split = train_test_split(df, test_size=0.4, random_state=1)
df_valid, df_test = train_test_split(df_valid_split, test_size=0.5, random_state=1)

features_train = df_train.drop('is_ultra', axis=1)
target_train = df_train['is_ultra']

features_valid = df_valid.drop('is_ultra', axis=1)
target_valid = df_valid['is_ultra']

features_test = df_test.drop('is_ultra', axis=1)
target_test = df_test['is_ultra']


#### Данные разделены на выборки как 0.6 на тренировочную, 0.2 на валидационную и 0.2 на тестовую.
#### Выделены Фичи и таргеты для каждой выборки

## 3. Исследуйте модели

In [3]:
for depth in range(1, 20): 
    model_tree_choise = DecisionTreeClassifier(max_depth=depth, random_state=1)
    model_tree_choise.fit(features_train, target_train)
    predictions_tree_ch_train = model_tree_choise.predict(features_train)
    predictions_tree_ch_valid = model_tree_choise.predict(features_valid)
    predictions_tree_ch_test = model_tree_choise.predict(features_test)
    print("max_depth =", depth, ": ")
    print('accuracy на тренировочной выборке', np.round(accuracy_score(target_train, predictions_tree_ch_train), 3))
    print('precision на тренировочной выборке', np.round(precision_score(target_train, predictions_tree_ch_train), 3))
    print('recall на тренировочной выборке', np.round(recall_score(target_train, predictions_tree_ch_train), 3))
    print()
    print('accuracy на валидационной выборке', np.round(accuracy_score(target_valid, predictions_tree_ch_valid), 3))
    print('precision на валидационной выборке', np.round(precision_score(target_valid, predictions_tree_ch_valid), 3))
    print('recall на валидационной выборке', np.round(recall_score(target_valid, predictions_tree_ch_valid), 3))
    print()
#    print('accuracy на тестовой выборке', np.round(accuracy_score(target_test, predictions_tree_ch_test), 3))
#    print()

max_depth = 1 : 
accuracy на тренировочной выборке 0.765
precision на тренировочной выборке 0.773
recall на тренировочной выборке 0.292

accuracy на валидационной выборке 0.719
precision на валидационной выборке 0.769
recall на валидационной выборке 0.231

max_depth = 2 : 
accuracy на тренировочной выборке 0.794
precision на тренировочной выборке 0.749
recall на тренировочной выборке 0.458

accuracy на валидационной выборке 0.756
precision на валидационной выборке 0.771
recall на валидационной выборке 0.389

max_depth = 3 : 
accuracy на тренировочной выборке 0.809
precision на тренировочной выборке 0.763
recall на тренировочной выборке 0.517

accuracy на валидационной выборке 0.771
precision на валидационной выборке 0.78
recall на валидационной выборке 0.444

max_depth = 4 : 
accuracy на тренировочной выборке 0.815
precision на тренировочной выборке 0.817
recall на тренировочной выборке 0.484

accuracy на валидационной выборке 0.768
precision на валидационной выборке 0.807
recall на ва

#### Был проварьирован гиперпараметр максимальной глубины от 1 до 20, оптимальными метриками обладает модель с глубиной 9, именно она принимается за удовлетворяющую техническому заданию. Меньше 9 имеется тенденция к недообучению, а более 9 начинает появляться переобучение модели

In [4]:
model_tree = DecisionTreeClassifier(max_depth=9, random_state=1)
model_tree.fit(features_train, target_train)
predictions_tree_train = model_tree.predict(features_train)
predictions_tree_valid = model_tree.predict(features_valid)
predictions_tree_test = model_tree.predict(features_test)
print("max_depth = 9")
print('accuracy на тренировочной выборке', np.round(accuracy_score(target_train, predictions_tree_train), 3))
print('precision на тренировочной выборке', np.round(precision_score(target_train, predictions_tree_train), 3))
print('recall на тренировочной выборке', np.round(recall_score(target_train, predictions_tree_train), 3))
print()
print('accuracy на валидационной выборке', np.round(accuracy_score(target_valid, predictions_tree_valid), 3))
print('precision на валидационной выборке', np.round(precision_score(target_valid, predictions_tree_valid), 3))
print('recall на валидационной выборке', np.round(recall_score(target_valid, predictions_tree_valid), 3))

max_depth = 9
accuracy на тренировочной выборке 0.883
precision на тренировочной выборке 0.958
recall на тренировочной выборке 0.635

accuracy на валидационной выборке 0.796
precision на валидационной выборке 0.815
recall на валидационной выборке 0.509


In [5]:
best_rand_model = None
best_rand_result = 0
for est in range(1, 10):
    model_rand = RandomForestClassifier(random_state=1, n_estimators=est) 
    model_rand.fit(features_train, target_train) 
    predictions_rand_train = model_rand.predict(features_train)
    predictions_rand_valid = model_rand.predict(features_valid)
    predictions_rand_test = model_rand.predict(features_test)
    
    result_rand_train = np.round(accuracy_score(target_train, predictions_rand_train), 3)
    result_rand_valid = np.round(accuracy_score(target_valid, predictions_rand_valid), 3)
    result_rand_test = np.round(accuracy_score(target_test, predictions_rand_test), 3)
    if result_rand_valid > best_rand_result:
        
        best_rand_model = model_rand 
        
        best_train_acc = result_rand_train
        best_train_prec = np.round(precision_score(target_train, predictions_rand_train), 3)
        best_train_recall = np.round(recall_score(target_train, predictions_rand_train), 3)
        
        best_valid_acc = result_rand_valid 
        best_valid_prec = np.round(precision_score(target_valid, predictions_rand_valid), 3)
        best_valid_recall = np.round(recall_score(target_valid, predictions_rand_valid), 3)
        
        best_test_acc = result_rand_test
        best_test_prec = np.round(precision_score(target_test, predictions_rand_test), 3)
        best_test_recall = np.round(recall_score(target_test, predictions_rand_test), 3)
        
        
        best_est = est
print('N_estimators лучшей модели:', best_est)
print("Accuracy наилучшей модели на тренировочной выборке:", best_train_acc)
print('precision наилучшей модели на тренировочной выборке', best_train_prec)
print('recall наилучшей модели на тренировочной выборке', best_train_recall)
print()
print("Accuracy  наилучшей модели на валидационной выборке:", best_valid_acc)
print('precision наилучшей модели на валидационной выборке:', best_valid_prec)
print('recall наилучшей модели на валидационной выборке:', best_valid_recall)

N_estimators лучшей модели: 9
Accuracy наилучшей модели на тренировочной выборке: 0.98
precision наилучшей модели на тренировочной выборке 0.978
recall наилучшей модели на тренировочной выборке 0.955

Accuracy  наилучшей модели на валидационной выборке: 0.784
precision наилучшей модели на валидационной выборке: 0.731
recall наилучшей модели на валидационной выборке: 0.565


#### Аналогично для модели случайного леса было проварьировано число оценщиков от 1 до 10, оптимальной признана модель с 9 оценщиками.

In [6]:
model_log = LogisticRegression(random_state=1) 
model_log.fit(features_train, target_train) 

predictions_log_train = model_log.predict(features_train)
predictions_log_valid = model_log.predict(features_valid)
predictions_log_test = model_log.predict(features_test)

print('accuracy на тренировочной выборке', np.round(accuracy_score(target_train, predictions_log_train), 3))
print('precision на тренировочной выборке', np.round(precision_score(target_train, predictions_log_train), 3))
print('recall на тренировочной выборке', np.round(recall_score(target_train, predictions_log_train), 3))
print()
print('accuracy на валидационной выборке', np.round(accuracy_score(target_valid, predictions_log_valid), 3))
print('precision на валидационной выборке', np.round(precision_score(target_valid, predictions_log_valid), 3))
print('recall на валидационной выборке', np.round(recall_score(target_valid, predictions_log_valid), 3))

accuracy на тренировочной выборке 0.751
precision на тренировочной выборке 0.896
recall на тренировочной выборке 0.18

accuracy на валидационной выборке 0.719
precision на валидационной выборке 0.889
recall на валидационной выборке 0.185


#### Модель логистической регрессии имеет не самые хорошиеметрики, и нет возможности что-то варьировать, поэтому данная модель не подойдет для нашей задачи ( точность менее 0.75)

## 4. Проверьте модель на тестовой выборке

In [7]:
print('accuracy модели дерева решений на тестовой выборке', np.round(accuracy_score(target_test, predictions_tree_test), 3))
print('precision модели дерева решений на тестовой выборке', np.round(precision_score(target_test, predictions_tree_test), 3))
print('recall модели дерева решений на тестовой выборке', np.round(recall_score(target_test, predictions_tree_test), 3))

accuracy модели дерева решений на тестовой выборке 0.787
precision модели дерева решений на тестовой выборке 0.746
recall модели дерева решений на тестовой выборке 0.462


In [8]:
print('accuracy модели случайного леса на тестовой выборке', np.round(accuracy_score(target_test, predictions_rand_test), 3))
print('precision модели случайного леса на тестовой выборке', np.round(precision_score(target_test, predictions_rand_test), 3))
print('recall модели случайного леса на тестовой выборке', np.round(recall_score(target_test, predictions_rand_test), 3))

accuracy модели случайного леса на тестовой выборке 0.798
precision модели случайного леса на тестовой выборке 0.708
recall модели случайного леса на тестовой выборке 0.579


In [9]:
print('accuracy модели логистической регрессии на тестовой выборке', np.round(accuracy_score(target_test, predictions_log_test), 3))
print('precision модели логистической регрессии на тестовой выборке', np.round(precision_score(target_test, predictions_log_test), 3))
print('recall модели логистической регрессии на тестовой выборке', np.round(recall_score(target_test, predictions_log_test), 3))

accuracy модели логистической регрессии на тестовой выборке 0.743
precision модели логистической регрессии на тестовой выборке 0.808
recall модели логистической регрессии на тестовой выборке 0.213


#### При проверке на тестовой выборке видно, что наилучшими показателями accuracy, precision и recall обладает модель случайного леса. Также хорошими показателями обладает модель дерева решений

## 5. (бонус) Проверьте модели на адекватность

In [10]:
from sklearn.dummy import DummyClassifier
model_dummy = DummyClassifier(random_state=1)
model_dummy.fit(features_train, target_train)
predictions_dummy_train = model_dummy.predict(features_train)
predictions_dummy_valid = model_dummy.predict(features_valid)
predictions_dummy_test = model_dummy.predict(features_test)

print('accuracy на тренировочной выборке', np.round(accuracy_score(target_train, predictions_dummy_train), 3))
print('precision на тренировочной выборке', np.round(precision_score(target_train, predictions_dummy_train), 3))
print('recall на тренировочной выборке', np.round(recall_score(target_train, predictions_dummy_train), 3))
print()
print('accuracy на валидационной выборке', np.round(accuracy_score(target_valid, predictions_dummy_valid), 3))
print('precision на валидационной выборке', np.round(precision_score(target_valid, predictions_dummy_valid), 3))
print('recall на валидационной выборке', np.round(recall_score(target_valid, predictions_dummy_valid), 3))
print()
print('accuracy на тестовой выборке', np.round(accuracy_score(target_test, predictions_dummy_test), 3))
print('precision на тестовой выборке', np.round(precision_score(target_test, predictions_dummy_test), 3))
print('recall на тестовой выборке', np.round(recall_score(target_test, predictions_dummy_test), 3))

accuracy на тренировочной выборке 0.583
precision на тренировочной выборке 0.302
recall на тренировочной выборке 0.309

accuracy на валидационной выборке 0.551
precision на валидационной выборке 0.315
recall на валидационной выборке 0.287

accuracy на тестовой выборке 0.571
precision на тестовой выборке 0.299
recall на тестовой выборке 0.299


#### Адекватность модели проверена сравнением с классификатором дурака (это мой авторский перевод :) ). Все метрики в моделях деревьев и случайного леса выше чем у данного классификатора, так что модель признана адекватной.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75


### В ходе вполнения проекта были рассмотрены несколько моделей машинного обучения и выбраны модели с наиболее хорошими метриками качества работы - модель случайного леса и дерево решений. Проведены варьирования гиперпараметров для них и отобраны наиболее подходящие. Сделаны промежуточные выводы.